### 'Assault', 'Robbery', and 'Sex Offences Forcible' categories. Let's call them 'Violent'
![](./img/img1.jpeg)
![](./img/img2.jpeg)
![](./img/img3.jpeg)
![](./img/img4.jpeg)
![](./img/img5.jpeg)
![](./img/img6.jpeg)

### This time, taking into account all the categories of crime
![](./img/img7.jpeg)

### добавить адрес
### как-то разбить Address, чтоб было меньше чем 23000 уникальных эл-ов
### если убрать Category, то множество строк будут одинаковыми, как бороться?
### можно добавить polynomial features от scikit

### добавить комментарии
### почистить в .py все лишнее

In [1]:
import pandas as pd
import numpy as np
import cvxopt
import cvxopt.solvers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from kernels import Kernel
import svm
import time
import copy

In [2]:
def accuracy(x, y):
    count = 0
    for i in range(len(x)):
        if x[i] == y[i]:
            count += 1
    return count/len(x)

In [6]:
train = pd.read_csv('/Users/grigoriipogorelov/Desktop/train.csv')
train['Date'] = pd.to_datetime(train['Dates'], errors='coerce')
train['Year'] = train['Date'].dt.year
train['Month'] = train['Date'].dt.month
train['WeekOfYear'] = train['Date'].dt.weekofyear
train['Day'] = train['Date'].dt.day
train['Time'] = train['Date'].dt.time
train['Hour'] = train['Date'].dt.hour
train = train.drop(['Dates','Date','Time','Resolution','Descript','Address'], axis=1)

cat_cols = ['DayOfWeek','PdDistrict', 'Year', 'Month', 'WeekOfYear', 'Day', 'Hour']
for cat in cat_cols:
    train = pd.get_dummies(train, columns=[cat])

In [5]:
train.Category.value_counts()

LARCENY/THEFT                  174900
OTHER OFFENSES                 126182
NON-CRIMINAL                    92304
ASSAULT                         76876
DRUG/NARCOTIC                   53971
VEHICLE THEFT                   53781
VANDALISM                       44725
WARRANTS                        42214
BURGLARY                        36755
SUSPICIOUS OCC                  31414
MISSING PERSON                  25989
ROBBERY                         23000
FRAUD                           16679
FORGERY/COUNTERFEITING          10609
SECONDARY CODES                  9985
WEAPON LAWS                      8555
PROSTITUTION                     7484
TRESPASS                         7326
STOLEN PROPERTY                  4540
SEX OFFENSES FORCIBLE            4388
DISORDERLY CONDUCT               4320
DRUNKENNESS                      4280
RECOVERED VEHICLE                3138
KIDNAPPING                       2341
DRIVING UNDER THE INFLUENCE      2268
RUNAWAY                          1946
LIQUOR LAWS 

In [7]:
train_len = len(train)
train_proportion = 0.001
train = train.sample(frac=1)
train = train.iloc[:int(len(train)*train_proportion)]
copy_ = copy.copy(train)
test = copy_.iloc[int(len(train)*0.8):]
train = train.iloc[:int(len(train)*0.8)]

categories = train.Category.unique()
category_len = len(train['Category'].unique())
samples = []
for i in range(category_len-1):
    for j in range(i+1, category_len):
        samples.append(categories[i] + '+' + categories[j])
        
train_sets = []
for sample in samples:
    cats = sample.split('+')
    train_0 = train.loc[train['Category'] == cats[0]]
    train_1 = train.loc[train['Category'] == cats[1]]
    frames = [train_0, train_1]
    train_sets.append(pd.concat(frames))
    
print(train.shape)

(702, 126)


In [8]:
start = time.time()

models = []
cat_to_number_each_model = []
C = 0.1
for i, data in enumerate(train_sets):
    _d = {}
    options = data['Category'].unique()
    data = data.as_matrix()
    data[data == options[0]] = 1
    _d['1'] = options[0]
    data[data == options[1]] = -1
    _d['-1'] = options[1]
    data = np.array(data, dtype='float')
    trainer = svm.SVMTrainer(kernel=Kernel.linear(), c=C)
    model = trainer.train(data[:,1:], data[:,0])
    models.append(model)
    cat_to_number_each_model.append(_d)
    print()
    print('MODEL '+str(i)+' finished')
    print()

end = time.time()
print(int((end - start)/60), ' minutes for training')

     pcost       dcost       gap    pres   dres
 0: -9.0458e+00 -2.6549e+01  8e+02  2e+01  2e-11
 1: -2.4202e+00 -2.3878e+01  6e+01  1e+00  2e-11
 2: -1.5216e+00 -7.7194e+00  6e+00  5e-16  5e-12
 3: -1.5642e+00 -3.0241e+00  1e+00  2e-16  3e-12
 4: -1.6247e+00 -1.9755e+00  4e-01  2e-16  3e-12
 5: -1.6664e+00 -1.7403e+00  7e-02  2e-16  3e-12
 6: -1.6825e+00 -1.6987e+00  2e-02  5e-16  3e-12
 7: -1.6874e+00 -1.6889e+00  1e-03  2e-16  4e-12
 8: -1.6880e+00 -1.6881e+00  4e-05  2e-16  4e-12
 9: -1.6880e+00 -1.6880e+00  9e-07  6e-16  4e-12
Optimal solution found.

MODEL 0 finished

     pcost       dcost       gap    pres   dres
 0: -2.6274e+00 -2.1571e+01  7e+02  2e+01  7e-12
 1: -7.8803e-01 -1.9111e+01  5e+01  1e+00  6e-12
 2: -4.9042e-01 -4.9593e+00  5e+00  9e-03  2e-12
 3: -4.9397e-01 -1.1636e+00  7e-01  1e-03  9e-13
 4: -4.9025e-01 -8.0075e-01  3e-01  2e-16  1e-12
 5: -5.0774e-01 -5.9250e-01  8e-02  2e-16  1e-12
 6: -5.2231e-01 -5.4748e-01  3e-02  2e-16  1e-12
 7: -5.2750e-01 -5.3285e-01 

 3: -1.9271e+00 -3.2535e+00  1e+00  5e-16  4e-12
 4: -2.0002e+00 -2.3892e+00  4e-01  3e-16  4e-12
 5: -2.0416e+00 -2.1191e+00  8e-02  9e-16  4e-12
 6: -2.0598e+00 -2.0719e+00  1e-02  6e-16  4e-12
 7: -2.0637e+00 -2.0644e+00  7e-04  8e-16  5e-12
 8: -2.0639e+00 -2.0640e+00  6e-05  6e-16  4e-12
 9: -2.0640e+00 -2.0640e+00  1e-06  1e-15  5e-12
Optimal solution found.

MODEL 14 finished

     pcost       dcost       gap    pres   dres
 0: -9.7283e+00 -2.5466e+01  8e+02  2e+01  2e-11
 1: -2.5342e+00 -2.2861e+01  6e+01  1e+00  1e-11
 2: -1.6512e+00 -7.3620e+00  6e+00  4e-04  5e-12
 3: -1.7231e+00 -2.7814e+00  1e+00  7e-05  3e-12
 4: -1.7942e+00 -2.0628e+00  3e-01  8e-06  3e-12
 5: -1.8292e+00 -1.8781e+00  5e-02  8e-08  3e-12
 6: -1.8416e+00 -1.8498e+00  8e-03  4e-09  3e-12
 7: -1.8443e+00 -1.8447e+00  4e-04  1e-10  3e-12
 8: -1.8445e+00 -1.8445e+00  7e-06  2e-12  3e-12
 9: -1.8445e+00 -1.8445e+00  1e-07  2e-14  3e-12
Optimal solution found.

MODEL 15 finished

     pcost       dcost       ga

     pcost       dcost       gap    pres   dres
 0: -8.5341e-01 -2.0094e+01  6e+02  2e+01  3e-12
 1: -2.9272e-01 -1.7711e+01  5e+01  1e+00  3e-12
 2: -1.6965e-01 -4.7951e+00  5e+00  1e-02  6e-13
 3: -1.6871e-01 -2.7439e-01  1e-01  2e-04  4e-13
 4: -1.7055e-01 -2.0685e-01  4e-02  5e-05  4e-13
 5: -1.7505e-01 -1.9386e-01  2e-02  2e-16  3e-13
 6: -1.7754e-01 -1.8209e-01  5e-03  2e-16  3e-13
 7: -1.7852e-01 -1.7962e-01  1e-03  1e-16  3e-13
 8: -1.7887e-01 -1.7899e-01  1e-04  2e-16  3e-13
 9: -1.7892e-01 -1.7892e-01  4e-06  2e-16  4e-13
10: -1.7892e-01 -1.7892e-01  9e-08  2e-16  3e-13
Optimal solution found.

MODEL 28 finished

     pcost       dcost       gap    pres   dres
 0: -6.7519e-01 -1.9681e+00  3e+01  6e+00  6e-13
 1: -5.8263e-01 -1.7446e+00  3e+00  4e-01  5e-13
 2: -4.4761e-01 -8.9630e-01  4e-01  7e-17  5e-13
 3: -4.5736e-01 -4.9449e-01  4e-02  6e-17  4e-13
 4: -4.6098e-01 -4.6346e-01  2e-03  6e-17  4e-13
 5: -4.6119e-01 -4.6125e-01  6e-05  5e-17  5e-13
 6: -4.6119e-01 -4.6119e-01


MODEL 43 finished

     pcost       dcost       gap    pres   dres
 0: -1.7558e+00 -3.0877e+00  4e+01  6e+00  4e-12
 1: -1.3357e+00 -2.6145e+00  3e+00  2e-01  2e-12
 2: -1.1020e+00 -1.4385e+00  3e-01  1e-16  2e-12
 3: -1.1251e+00 -1.1497e+00  2e-02  2e-16  1e-12
 4: -1.1296e+00 -1.1304e+00  8e-04  1e-16  2e-12
 5: -1.1299e+00 -1.1299e+00  1e-05  1e-16  2e-12
 6: -1.1299e+00 -1.1299e+00  1e-07  8e-17  1e-12
Optimal solution found.

MODEL 44 finished

     pcost       dcost       gap    pres   dres
 0: -1.5131e+00 -3.5160e+00  5e+01  7e+00  2e-12
 1: -1.2618e+00 -3.1159e+00  3e+00  2e-01  2e-12
 2: -1.1575e+00 -1.4759e+00  3e-01  1e-16  1e-12
 3: -1.2094e+00 -1.2332e+00  2e-02  8e-17  2e-12
 4: -1.2190e+00 -1.2205e+00  1e-03  1e-16  1e-12
 5: -1.2199e+00 -1.2199e+00  3e-05  2e-16  1e-12
 6: -1.2199e+00 -1.2199e+00  3e-07  8e-17  2e-12
Optimal solution found.

MODEL 45 finished

     pcost       dcost       gap    pres   dres
 0: -1.0125e+00 -2.6700e+00  4e+01  6e+00  1e-12
 1: -9.1627e-

Optimal solution found.

MODEL 62 finished

     pcost       dcost       gap    pres   dres
 0: -5.3954e-01 -1.7240e+00  3e+01  5e+00  8e-13
 1: -5.0010e-01 -1.5475e+00  2e+00  3e-01  8e-13
 2: -4.1939e-01 -7.5566e-01  3e-01  9e-17  6e-13
 3: -4.2746e-01 -4.4682e-01  2e-02  6e-17  6e-13
 4: -4.2956e-01 -4.3014e-01  6e-04  4e-17  5e-13
 5: -4.2961e-01 -4.2961e-01  6e-06  6e-17  9e-13
 6: -4.2961e-01 -4.2961e-01  6e-08  6e-17  8e-13
Optimal solution found.

MODEL 63 finished

     pcost       dcost       gap    pres   dres
 0: -1.4240e+00 -7.3973e+00  2e+02  1e+01  4e-12
 1: -8.2890e-01 -6.4257e+00  2e+01  1e+00  3e-12
 2: -4.8288e-01 -2.4221e+00  2e+00  2e-16  2e-12
 3: -5.0127e-01 -7.8489e-01  3e-01  1e-16  1e-12
 4: -5.2221e-01 -5.4790e-01  3e-02  2e-16  1e-12
 5: -5.2786e-01 -5.3150e-01  4e-03  1e-16  1e-12
 6: -5.2906e-01 -5.2932e-01  3e-04  1e-16  1e-12
 7: -5.2917e-01 -5.2917e-01  4e-06  1e-16  1e-12
 8: -5.2917e-01 -5.2917e-01  5e-08  1e-16  1e-12
Optimal solution found.

MODEL 6

...

 4: -2.3152e+00 -2.3465e+00  3e-02  4e-16  3e-12
 5: -2.3244e+00 -2.3276e+00  3e-03  2e-16  3e-12
 6: -2.3254e+00 -2.3255e+00  2e-04  2e-16  4e-12
 7: -2.3254e+00 -2.3254e+00  7e-06  2e-16  4e-12
 8: -2.3254e+00 -2.3254e+00  5e-07  2e-16  3e-12
Optimal solution found.

MODEL 308 finished

     pcost       dcost       gap    pres   dres
 0: -1.9845e+00 -6.3013e+00  1e+02  1e+01  2e-12
 1: -1.2949e+00 -5.4707e+00  1e+01  8e-01  2e-12
 2: -8.0668e-01 -2.4992e+00  2e+00  2e-16  1e-12
 3: -8.3705e-01 -1.1252e+00  3e-01  9e-17  8e-13
 4: -8.6150e-01 -9.0940e-01  5e-02  1e-16  8e-13
 5: -8.7050e-01 -8.7339e-01  3e-03  1e-16  8e-13
 6: -8.7139e-01 -8.7167e-01  3e-04  1e-16  9e-13
 7: -8.7151e-01 -8.7152e-01  1e-05  1e-16  9e-13
 8: -8.7151e-01 -8.7151e-01  1e-06  1e-16  9e-13
 9: -8.7151e-01 -8.7151e-01  8e-08  1e-16  1e-12
Optimal solution found.

MODEL 309 finished

     pcost       dcost       gap    pres   dres
 0: -8.3410e-01 -5.0693e+00  1e+02  1e+01  2e-12
 1: -5.6116e-01 -4.3947e+00  1

Optimal solution found.

MODEL 326 finished

     pcost       dcost       gap    pres   dres
 0: -1.0155e+00 -8.7452e+00  2e+02  1e+01  2e-12
 1: -5.9230e-01 -7.6234e+00  2e+01  1e+00  2e-12
 2: -3.2989e-01 -2.7537e+00  2e+00  2e-16  9e-13
 3: -3.3897e-01 -6.9112e-01  4e-01  1e-16  5e-13
 4: -3.4683e-01 -4.3390e-01  9e-02  2e-16  7e-13
 5: -3.5196e-01 -3.6855e-01  2e-02  2e-16  6e-13
 6: -3.5527e-01 -3.5834e-01  3e-03  1e-16  5e-13
 7: -3.5614e-01 -3.5656e-01  4e-04  1e-16  5e-13
 8: -3.5629e-01 -3.5630e-01  1e-05  1e-16  7e-13
 9: -3.5630e-01 -3.5630e-01  6e-07  2e-16  8e-13
10: -3.5630e-01 -3.5630e-01  2e-08  1e-16  8e-13
Optimal solution found.

MODEL 327 finished

     pcost       dcost       gap    pres   dres
 0: -6.1363e-01 -8.5866e+00  2e+02  1e+01  1e-12
 1: -3.0594e-01 -7.5058e+00  2e+01  1e+00  1e-12
 2: -1.6840e-01 -2.3707e+00  2e+00  7e-03  4e-13
 3: -1.7049e-01 -2.1250e-01  4e-02  1e-04  3e-13
 4: -1.7624e-01 -1.8930e-01  1e-02  1e-16  3e-13
 5: -1.7764e-01 -1.7881e-01  1

     pcost       dcost       gap    pres   dres
 0: -1.4569e+00 -3.6965e+00  6e+01  8e+00  3e-12
 1: -1.2158e+00 -3.3084e+00  5e+00  3e-01  3e-12
 2: -1.0897e+00 -1.6825e+00  6e-01  2e-16  1e-12
 3: -1.1513e+00 -1.2156e+00  6e-02  8e-17  2e-12
 4: -1.1718e+00 -1.1764e+00  5e-03  9e-17  1e-12
 5: -1.1734e+00 -1.1736e+00  3e-04  1e-16  2e-12
 6: -1.1735e+00 -1.1735e+00  2e-05  3e-16  1e-12
 7: -1.1735e+00 -1.1735e+00  5e-07  2e-16  2e-12
Optimal solution found.

MODEL 345 finished

     pcost       dcost       gap    pres   dres
 0: -1.9732e+00 -3.4730e+00  6e+01  7e+00  2e-12
 1: -1.3579e+00 -2.9785e+00  4e+00  3e-01  2e-12
 2: -1.1025e+00 -1.5830e+00  5e-01  9e-17  2e-12
 3: -1.1330e+00 -1.1879e+00  5e-02  5e-17  1e-12
 4: -1.1426e+00 -1.1463e+00  4e-03  1e-16  1e-12
 5: -1.1437e+00 -1.1438e+00  1e-04  2e-16  2e-12
 6: -1.1437e+00 -1.1437e+00  1e-06  2e-16  1e-12
Optimal solution found.

MODEL 346 finished

     pcost       dcost       gap    pres   dres
 0: -7.1206e-01 -2.0441e+00  3e


     pcost       dcost       gap    pres   dres
 0: -9.4786e-01 -2.4137e+00  4e+01  6e+00  9e-13
 1: -7.2882e-01 -2.0929e+00  5e+00  5e-01  1e-12
 2: -4.8420e-01 -1.1192e+00  6e-01  9e-17  5e-13
 3: -4.9689e-01 -5.7004e-01  7e-02  6e-17  3e-13
 4: -5.0290e-01 -5.0901e-01  6e-03  5e-17  4e-13
 5: -5.0358e-01 -5.0386e-01  3e-04  7e-17  4e-13
 6: -5.0364e-01 -5.0365e-01  2e-05  5e-17  3e-13
 7: -5.0365e-01 -5.0365e-01  9e-07  8e-17  4e-13
 8: -5.0365e-01 -5.0365e-01  1e-08  7e-17  4e-13
Optimal solution found.

MODEL 370 finished

     pcost       dcost       gap    pres   dres
 0: -3.4080e-01 -1.6789e+00  4e+01  6e+00  5e-13
 1: -2.7452e-01 -1.4537e+00  4e+00  5e-01  4e-13
 2: -1.6097e-01 -6.2595e-01  5e-01  1e-16  3e-13
 3: -1.6712e-01 -2.2999e-01  6e-02  6e-17  2e-13
 4: -1.7181e-01 -1.7814e-01  6e-03  6e-17  2e-13
 5: -1.7245e-01 -1.7293e-01  5e-04  6e-17  2e-13
 6: -1.7253e-01 -1.7257e-01  4e-05  6e-17  2e-13
 7: -1.7254e-01 -1.7255e-01  3e-06  5e-17  2e-13
 8: -1.7254e-01 -1.7254e-

 5: -1.5286e-01 -1.5286e-01  2e-07  2e-17  1e-13
 6: -1.5286e-01 -1.5286e-01  2e-09  2e-17  8e-14
Optimal solution found.

MODEL 404 finished

     pcost       dcost       gap    pres   dres
 0: -1.6156e-01 -3.4843e-01  5e+00  2e+00  4e-13
 1: -1.4316e-01 -3.0498e-01  2e-01  3e-02  3e-14
 2: -1.3673e-01 -1.4537e-01  9e-03  4e-17  1e-13
 3: -1.3991e-01 -1.4005e-01  1e-04  2e-17  6e-14
 4: -1.4000e-01 -1.4000e-01  1e-06  0e+00  8e-14
 5: -1.4000e-01 -1.4000e-01  1e-08  4e-17  2e-14
Optimal solution found.

MODEL 405 finished

     pcost       dcost       gap    pres   dres
 0: -1.6153e-01 -3.4842e-01  5e+00  2e+00  4e-14
 1: -1.4315e-01 -3.0497e-01  2e-01  3e-02  1e-13
 2: -1.3672e-01 -1.4535e-01  9e-03  2e-17  3e-14
 3: -1.3990e-01 -1.4004e-01  1e-04  0e+00  1e-13
 4: -1.3999e-01 -1.3999e-01  1e-06  4e-17  1e-13
 5: -1.3999e-01 -1.3999e-01  1e-08  2e-17  4e-14
Optimal solution found.

MODEL 406 finished

     pcost       dcost       gap    pres   dres
 0: -4.1736e-01 -1.9968e+00  5e+01 

In [9]:
labels = test['Category'].as_matrix()
test = test.as_matrix()
test = np.array(test[:,1:], dtype='float')

start = time.time()

# should stick to 'samples' order
predictions_for_each_model = []
for model in models:
    pred = []
    for t in test:
        pred.append(model.predict(t))
    predictions_for_each_model.append(pred)
    
end = time.time()
print(int((end - start)/60), ' minutes for predicting')

0  minutes for predicting


In [11]:
count_for_categories = {}
for cat in categories:
    count_for_categories[cat] = 0

combo_predictions = []
pred_len = len(predictions_for_each_model[0])

# for every prediction of a model
for i in range(pred_len):
    # take a particular model
    for j, preds in enumerate(predictions_for_each_model):
        count_cat = copy.deepcopy(count_for_categories)
        count_cat[cat_to_number_each_model[j][str(int(preds[i]))]] += 1
    winner = max(count_cat, key=count_cat.get)
    combo_predictions.append(winner)

### поиграться с С

In [12]:
print('accuracy is ', accuracy(labels,combo_predictions))

accuracy is  0.0013287775246772968
